#🧠V1_Animagine XL_3.0_Navez ⚡

*if you find this helpful consider becoming a member on patreon, subscribe to my youtube for Ai applications guides*

[![Patreon](https://img.shields.io/badge/Patreon-Support-orange?style=rounded-square&logo=patreon)](https://www.patreon.com/user?u=98917275)
[![Youtube](https://img.shields.io/badge/Youtube-Subscribe-red?style=rounded-square&logo=youtube)](https://www.youtube.com/@clementetumbajulcan4712)
[![GitHub](https://img.shields.io/badge/GitHub-Visit-blue?style=rounded-square&logo=github)](https://github.com/Navezjt)
[![Huggingface](https://img.shields.io/badge/Huggingface-visit-yellow?style=rounded-square&logo=huggingface)](https://huggingface.co/JCTN)
[![Instagram](https://img.shields.io/badge/Instagarm-Follow-pink?style=rounded-square&logo=Instagram)](https://Instagram.com/joeltumbajulca)


# Animagine XL 3.0 : A Quick-Eyed Sky Colab

Model card: https://huggingface.co/cagliostrolab/animagine-xl-3.0

Animagine XL 3.0 is an advanced latent text-to-image diffusion model designed to create high-resolution, detailed anime images. It's fine-tuned from Stable Diffusion XL 1.0 using a high-quality anime-style image dataset. This model, an upgrade from Animagine XL 1.0, excels in capturing the diverse and distinct styles of anime art, offering improved image quality and aesthetics.<br>
<br>

The Quick-Eyed Sky fork introduces several key advancements:<b>

- Integration of a feature for directly saving generated images to your Google Drive.

- Implementation of Dynamic Prompting, a technique that generates varied outputs by randomly selecting from structured lists (e.g., "Photo of a {car|train|truck} in the {sun|rain|evening|mist}").

- Automated preservation of settings for each generation, including Dynamic Prompts, in a separate .txt file.

- Enhanced batch processing capabilities, allowing for up to 200 images in a single session.

- Expanded image dimension support up to 2048 pixels. Starting from 1024 pixels is recommended.

- An upgraded Gradio interface, offering an enriched and intuitive user experience.</b>

This works well all a T4.</b><br><br>

<b>Upon completion of the installation, access the tool via the provided "Running on public URL" link.</b><br>

<br>Enjoy!<br><br>

Support on Patreon: <b>https://www.patreon.com/Quick_Eyed_Sky</b><br>
Explore on YouTube: <b>https://www.youtube.com/@QuickEyedSky </b><br><br>



<img src="https://raw.githubusercontent.com/Quick-Eyed-Sky/Images/main/5c1f7263-47fe-433d-bc5a-e56b4ab4f981.png" width="80%" alt="Cosmocat">

In [ ]:
#@title # <font color="#0497e0">**JOEL CTN**</font>UI + <font color="#4684a3"> RADIO
%%html
<b>Press play on the music player(Uses only 13MB of data)</b><br/>
<audio src="http://stream.zeno.fm/vq4s9m2sg48uv" controls>

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-b45e668c-45f7-90b3-8f15-249f394d80f5)


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#@title # <font color="#0497e0">**JOEL CTN**</font>UI + <font color="#4684a3"> Installation
# Install the required packages for Stable Diffusion and Animagine XL 3.0
!pip install -q --upgrade diffusers transformers gradio torch safetensors accelerate
!pip install -q scipy ftfy

import torch
import numpy as np
import os
import random
import uuid
from PIL import Image
import gradio as gr

# Check the installed version of diffusers
!pip show diffusers

# Importing the classes for Animagine XL 3.0 model
from diffusers import (
    StableDiffusionXLPipeline,
    EulerAncestralDiscreteScheduler,
    AutoencoderKL
)

# Load the VAE component specific to Animagine XL 3.0
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

# Configure the pipeline for Animagine XL 3.0
# Note: Removed the 'variant="fp16"' argument based on the error encountered
pipe = StableDiffusionXLPipeline.from_pretrained(
    "cagliostrolab/animagine-xl-3.0",
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True
)

# Use the Euler Ancestral Discrete Scheduler for improved image generation
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to('cuda')

# Create a new folder for the Animagine XL 3.0 project
new_folder_path = "/content/drive/My Drive/Animagine-XL-3-0-Images"  # Update the folder name as needed

# Create the folder if it doesn't exist
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)



In [ ]:
#@title Image Generation Settings
DESCRIPTION = """# QES's Animagine XL 3.0 Integration"""
if not torch.cuda.is_available():
    DESCRIPTION += "\n<p>Running on CPU 🥶 This demo may not work on CPU.</p>"

MAX_SEED = np.iinfo(np.int32).max
CACHE_EXAMPLES = torch.cuda.is_available() and os.getenv("CACHE_EXAMPLES", "1") == "1"
MAX_IMAGE_SIZE = int(os.getenv("MAX_IMAGE_SIZE", "2048"))
USE_TORCH_COMPILE = os.getenv("USE_TORCH_COMPILE", "0") == "1"
ENABLE_CPU_OFFLOAD = os.getenv("ENABLE_CPU_OFFLOAD", "0") == "1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the Animagine XL 3.0 pipeline
model_id = "cagliostrolab/animagine-xl-3.0"
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

# Configuring the pipeline without specifying the 'variant' argument
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True
).to(device)

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)


# Function to save images
def save_image(img, folder_path):
    unique_name = str(uuid.uuid4()) + ".png"
    full_path = os.path.join(folder_path, unique_name)
    img.save(full_path)
    return full_path

# Function to process dynamic prompts
def process_dynamic_prompt(prompt):
    while '{' in prompt:
        start = prompt.find('{')
        end = prompt.find('}', start)
        if end == -1:
            break
        options = prompt[start+1:end].split('|')
        choice = random.choice(options)
        prompt = prompt[:start] + choice + prompt[end+1:]
    return prompt

# Function to check for dynamic elements in prompts
def contains_dynamic_elements(prompt):
    return '{' in prompt and '}' in prompt

# Function to randomize seed
def randomize_seed_fn(seed: int, randomize_seed: bool) -> int:
    if randomize_seed:
        seed = random.randint(0, MAX_SEED)
    return seed

def save_individual_settings(folder_path, image_num, individual_prompt, original_dynamic_prompt, negative_prompt, seed, width, height, guidance_scale, randomize_seed, steps):
    try:
        filename = f"settings_image_{image_num}_{str(uuid.uuid4())}.txt"
        full_path = os.path.join(folder_path, filename)
        with open(full_path, 'w') as file:
            file.write(f"Original Dynamic Prompt: {original_dynamic_prompt}\n")
            file.write(f"Processed Prompt: {individual_prompt}\n")
            file.write(f"Negative Prompt: {negative_prompt if negative_prompt else 'None'}\n")
            file.write(f"Seed: {seed}\n")
            file.write(f"Width: {width}\n")
            file.write(f"Height: {height}\n")
            file.write(f"Guidance Scale: {guidance_scale}\n")
            file.write(f"Randomize Seed: {'Yes' if randomize_seed else 'No'}\n")
            file.write(f"Steps: {steps}\n")
        print(f"Settings for image {image_num} saved to {full_path}")
    except Exception as e:
        print(f"Error saving settings for image {image_num}: {e}")

def save_settings(folder_path, prompt, negative_prompt, seed, width, height, guidance_scale, num_images, randomize_seed, steps):
    try:
        unique_id = str(uuid.uuid4())
        filename = f"settings_{unique_id}.txt"
        full_path = os.path.join(folder_path, filename)
        with open(full_path, 'w') as file:
            file.write(f"Prompt: {prompt}\n")
            file.write(f"Negative Prompt: {negative_prompt if negative_prompt else 'None'}\n")
            file.write(f"Seed: {seed}\n")
            file.write(f"Width: {width}\n")
            file.write(f"Height: {height}\n")
            file.write(f"Guidance Scale: {guidance_scale}\n")
            file.write(f"Number of Images: {num_images}\n")
            file.write(f"Randomize Seed: {randomize_seed}\n")
            file.write(f"Steps: {steps}\n")
        print(f"Settings saved to {full_path}")
    except Exception as e:
        print(f"Error saving batch settings: {e}")

# Image generation function adapted for Animagine XL 3.0
def generate(
    prompt: str,
    negative_prompt: str = "",
    use_negative_prompt: bool = False,
    seed: int = 0,
    width: int = 1024,
    height: int = 1024,
    guidance_scale: float = 7,
    randomize_seed: bool = False,
    num_images: int = 1,
    steps: int = 25,
    progress=gr.Progress(track_tqdm=True),
):
    # Update folder path as per installation cell for Animagine XL 3.0
    folder_path = "/content/drive/My Drive/Animagine-XL-3-0-Images"

    pipe.to(device)
    seed = int(randomize_seed_fn(seed, randomize_seed))
    generator = torch.Generator().manual_seed(seed)

    if not use_negative_prompt:
        negative_prompt = None

    dynamic_prompt = contains_dynamic_elements(prompt)
    if not dynamic_prompt:
        save_settings(folder_path, prompt, negative_prompt, seed, width, height, guidance_scale, num_images, randomize_seed, steps)

    image_paths = []
    for i in range(num_images):
        individual_prompt = process_dynamic_prompt(prompt) if dynamic_prompt else prompt
        neg_prompt = negative_prompt if use_negative_prompt else None

        images = pipe(prompt=individual_prompt, negative_prompt=neg_prompt, width=width, height=height, guidance_scale=guidance_scale, num_inference_steps=steps, generator=generator).images

        for img in images:
            image_path = save_image(img, folder_path)
            image_paths.append(image_path)

            if dynamic_prompt:
                save_individual_settings(folder_path, i+1, individual_prompt, prompt, negative_prompt, seed, width, height, guidance_scale, randomize_seed, steps)

    return image_paths, seed



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
#@title # <font color="#0497e0">**JOEL CTN**</font>UI + <font color="#4684a3"> Gradio Settings (when ready, click on the link "Running on Public URL")

css = '''
.gradio-container {
    max-width: 1680px !important;
}
h1 {
    text-align: center;
}
'''
with gr.Blocks(css=css) as demo:
    gr.Markdown(DESCRIPTION)
    gr.DuplicateButton(
        value="Duplicate Space for private use",
        elem_id="duplicate-button",
        visible=os.getenv("SHOW_DUPLICATE_BUTTON") == "1",
    )
    with gr.Group():
        with gr.Row():
            prompt = gr.Text(
                label="Prompt",
                show_label=False,
                placeholder="Enter your prompt",
                container=False,
            )
            run_button = gr.Button("Run", scale=0)
        result = gr.Gallery(label="Result", show_label=False)

    with gr.Accordion("Advanced options", open=True):
        with gr.Row():
            use_negative_prompt = gr.Checkbox(label="Use negative prompt", value=True)
            negative_prompt = gr.Text(
                label="Negative prompt",
                max_lines=10,
                placeholder="Enter a negative prompt",
                visible=True,
            )
        seed = gr.Slider(
            label="Seed",
            minimum=0,
            maximum=MAX_SEED,
            step=1,
            value=0,
        )
        randomize_seed = gr.Checkbox(label="Randomize seed", value=True)
        with gr.Row(visible=True):
            width = gr.Slider(
                label="Width",
                minimum=256,
                maximum=MAX_IMAGE_SIZE,
                step=32,
                value=1024,
            )
            height = gr.Slider(
                label="Height",
                minimum=256,
                maximum=MAX_IMAGE_SIZE,
                step=32,
                value=1024,
            )
        with gr.Row():
            guidance_scale = gr.Slider(
                label="Guidance Scale",
                minimum=0.1,
                maximum=20,
                step=0.1,
                value=9,
            )
            steps = gr.Slider(minimum=1, maximum=100, step=1, label="Steps", value=40)  # Slider for steps
        num_images = gr.Slider(minimum=1, maximum=200, step=1, label="Number of Images", value=6)  # Moved to the end

    use_negative_prompt.change(
        fn=lambda x: gr.update(visible=x),
        inputs=use_negative_prompt,
        outputs=negative_prompt,
        api_name=False,
    )

    gr.on(
        triggers=[
            prompt.submit,
            negative_prompt.submit,
            run_button.click,
        ],
        fn=generate,
        inputs=[
            prompt,
            negative_prompt,
            use_negative_prompt,
            seed,
            width,
            height,
            guidance_scale,
            randomize_seed,
            num_images,  # Ensure 'num_images' is in the correct order
            steps,  # Ensure 'steps' is in the correct order
        ],
        outputs=[result, seed],
        api_name="run",
    )

if __name__ == "__main__":
    demo.queue(max_size=20).launch(inline=False)



My Colabs:<br><br>

•	Animagine XL 2.0: https://colab.research.google.com/drive/1U9L8MinRDl6uF6ia-6N1URIfzXpy3SGS?usp=sharing

•	SDXL-DPO: https://colab.research.google.com/drive/15f1fc6krTHaRPCRhUpWz6M0V1mnyOZtQ?usp=sharing

•	Dreamlike Photoreal 2.0: https://colab.research.google.com/drive/1ZjbbuledqapwnXVOFc6RxmTgtPi3Dd3Z?usp=sharing

•	Stable Diffusion SDXL + Kandinsky 3 + Playground V2: https://colab.research.google.com/drive/16ptLGQvzVy8V_LdNe0d0hADSQpjuUJ62?usp=sharing

•	PlayGround V2: https://colab.research.google.com/drive/1fXyZ3KiZpmUa5LVa1Vw9EfKoYtbkZLah?usp=sharing

•	Stable Video Diffusion: https://colab.research.google.com/drive/1y-vVm_PhDjpI6E_Yo6yVxYkdAfjm-6U0?usp=sharing

•	AudioCraft MusicGen: https://colab.research.google.com/drive/1eHbMw7yTvTH_8oelN86ES55upGXa2KN3?usp=sharing

•	QES MusicGEN: https://colab.research.google.com/drive/1eHbMw7yTvTH_8oelN86ES55upGXa2KN3?usp=sharing

•	GFPGAN Face Restoration: https://colab.research.google.com/drive/1ypBZ8MGFqXz3Vte-yuvCTHL-t3xTLuD3?usp=sharing

•	Image-to-Text-to-Image: https://colab.research.google.com/drive/1zsG6ruSF_Rk8Yw8OeqA0746_GVjV8Y18?usp=sharing

•	Kandinsky 2.1 + Symmetry + Dynamic Prompting + CSV + Upscaling: https://colab.research.google.com/drive/18QU-dzz4HzUcXVgVxFP_Ck1qDutXOPrv?usp=sharing

•	Kandinsky 2.1 + Automatic Outpainting: https://colab.research.google.com/drive/1e9jGVEGnvSaHqKdjzehhzSz-d4kRRYVh?usp=sharing

•	Deforum 7 + Dynamic Prompting: https://colab.research.google.com/drive/1hNmWnXnoNE9vXTujD3udJRSYTjSjowvV?usp=sharing

•	Kandinsky 3: https://colab.research.google.com/drive/13sLgeMJ_-zP7a3LIWmBEOQY3ZIk_Lhuh?usp=sharing
